In [0]:
%run /Workspace/Users/dataeng.victor_outlook.com#ext#@dataengvictoroutlook.onmicrosoft.com/01.config/00.global_config


In [0]:
%run /Workspace/Repos/dataeng.victor@outlook.com/databricks-weather-realtime/04.gold/01_gold_hourly_metrics

In [0]:
from pyspark.sql.functions import *
from pyspark.sql import Window

silver_table = f"{catalog}.{silver['schema']}.{silver['table']}"

df = spark.read.table(silver_table)

df_hour = df.withColumn(
    "window_hour",
    date_trunc("hour", col("timestamp"))
)



In [0]:
df_gold = (
    df_hour.groupBy("window_hour")
    .agg(
        avg("temperature").alias("temp_avg"),
        min("temperature").alias("temp_min"),
        max("temperature").alias("temp_max"),

        avg("humidity").alias("humidity_avg"),

        avg("wind_speed").alias("wind_speed_avg"),
        max("wind_gust").alias("wind_gust_max"),
        
        avg("visibility").alias("visibility_avg"),
        
        avg("pressure_sea_level").alias("pressure_avg")
    )
    .orderBy("window_hour")
)


In [0]:
#SCHEMA GOLD
gold = {
    "schema": "gold",
    "table_hourly": "weather_hourly_metrics"
}


In [0]:
#Escrever em delta os dados agregados
df_gold.write.mode("overwrite").format("delta").saveAsTable(
    f"{catalog}.gold.weather_hourly_metrics"
)

In [0]:
%sql
SELECT * FROM weather_realtime.gold.weather_hourly_metrics ORDER BY window_hour;
